In [ ]:
# This notebook consists of actions for preparation for SeismicGeometry tests
# It creates a fake cube as random noise and convert it into available data formats
import os
import sys
from datetime import date
import shutil
import warnings
warnings.filterwarnings("ignore")

import numpy as np

sys.path.append('../../../seismiqb')

from seismiqb import SeismicGeometry
from seismiqb.geometry import export

In [ ]:
# Workspace constants
DATESTAMP = date.today().strftime("%Y-%m-%d")
TESTS_ROOT_DIR = './'

# Data creation parameters
CUBE_NAME = f'test_cube_{DATESTAMP}.sgy'
CUBE_SHAPE = (1000, 200, 400)
SEED = 42

# Data formats for which running tests
FORMATS = ['sgy', 'hdf5', 'qhdf5', 'blosc', 'qblosc']

# Prepare a workspace

Create directories for files and create a fake cube and save it.

**Storage structure:**
___


**geometry_test_files** (tests root directory)

&emsp;├── **test_array.npy**

&emsp;└── **test_cube.{DATAFORMAT}**

---
 * **{DATAFORMAT}** means each data format from **FORMATS** list

In [ ]:
OUTPUT_DIR = os.path.join(TESTS_ROOT_DIR, "geometry_test_files")

# (Re)create the test directory
if os.path.exists(OUTPUT_DIR):
    shutil.rmtree(OUTPUT_DIR)

os.makedirs(OUTPUT_DIR)

In [ ]:
CUBE_PATH = os.path.join(OUTPUT_DIR, CUBE_NAME)
TEST_PATH_DB = os.path.join(OUTPUT_DIR, 'geometry_test_db')

# Create a fake cube

In [ ]:
rng = np.random.default_rng(SEED)
data_array = rng.normal(0, 1000, CUBE_SHAPE).astype(np.float32)

In [ ]:
with open(os.path.join(OUTPUT_DIR, f'test_array_{DATESTAMP}.npy'), 'wb') as outfile:
    np.save(outfile, data_array)

In [ ]:
%%time
export.make_segy_from_array(
    array=data_array, path_segy=CUBE_PATH, zip_segy=False,
    sample_rate=2., delay=50, pbar='t'
)

In [ ]:
%%time
geometry_sgy = SeismicGeometry(
    path=CUBE_PATH, process=True, collect_stats=True, spatial=True, pbar='t'
)

# Check data loading

In [ ]:
%%time
_ = SeismicGeometry(
    path=CUBE_PATH,
    headers=SeismicGeometry.HEADERS_POST_FULL,
    index_headers=SeismicGeometry.INDEX_CDP
)

In [ ]:
%%time
_ = SeismicGeometry(
    path=CUBE_PATH,
    headers=SeismicGeometry.HEADERS_POST_FULL,
    index_headers=SeismicGeometry.INDEX_POST
)

# Conversion

In [ ]:
for f in FORMATS:
    if f != 'sgy':
        _ = geometry_sgy.convert(format=f, quantize=False, store_meta=False, pbar='t')